In [75]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np
import json
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import light_location_dict

In [76]:
level1_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level1.json")))

In [77]:
level2_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level2.json")))

In [78]:
level3_json = json.load(
    open(Db.get_project_path("Project/_04TPMAlgorithm/TPM/output/NZERTF_year1_minsup0.14_minconf_0.5/level3.json")))

In [79]:
meta = Db.load_data(meta=True, consumption=False, hourly=False)

In [80]:
def json_to_dataframe(json_file):
    if "," in json_file[0]["name_node"]:
        level_df = pd.DataFrame(columns=["pattern","supp","conf", "time"])
        for i in json_file:
            for j in i["patterns"]:
                level_df.loc[level_df.shape[0]] = j
    else:
        level_df = pd.DataFrame(columns=["name_node","supp","conf"])
        for i in json_file:
            level_df.loc[level_df.shape[0]] = i
    return level_df

In [81]:
level2_df = json_to_dataframe(level2_json)

In [82]:
level3_df = json_to_dataframe(level3_json)
level3_df

,pattern,supp,conf,time
0,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.698630,0.704420,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
1,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
2,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.701370,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
3,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
4,Load_StatusMBALights->Load_StatusMBALights*Loa...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
...,...,...,...,...
5767,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.707182,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5768,Load_StatusPlugLoadHeatingPad>Load_StatusMBALi...,0.701370,0.701370,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5769,Load_StatusPlugLoadHeatingPad->Load_StatusSens...,0.698630,0.704420,"{'134': [[['2013-11-12 21:59:02', '2013-11-12 ..."
5770,Load_StatusApplianceOven>Load_StatusPlugLoadVi...,0.276712,0.651613,"{'180': [[['2013-12-28 08:14:06', '2013-12-28 ..."


In [83]:
level3_df_follow_restricted = level3_df.loc[level3_df['pattern'].str.findall('-').map(len) <= 1]

In [87]:
person_appliance_rule_indexes = []
appliance_appliance_rule_indexes = []
for index, row in level3_df_follow_restricted.iterrows():
    tmp_floor_set = set()
    appliance_check_list = list()
    person_check_list = list()
    for col in set(re.findall('[\w_]+', row['pattern'])):
        person_check_list.append('SensHeat' in col)
        appliance_check_list.append('SensHeat' not in col)
        try:
            tmp_floor_set.add(meta.loc[col, 'Measurement_Floor'])
        except:
            tmp_floor_set.add(light_location_dict(meta)[col][0])
    if '1stFloor' in tmp_floor_set and '2ndFloor' in tmp_floor_set and sum(appliance_check_list) >= 1:
        attribute_list = set(re.findall('[\w_]+', row['pattern']))
        if sum(person_check_list) >= 1:
            person_appliance_rule_indexes.append(index)
        else:
            appliance_appliance_rule_indexes.append(index)
print(len(person_appliance_rule_indexes))

1577


In [94]:
level3_df_follow_restricted.loc[(level3_df_follow_restricted.loc[person_appliance_rule_indexes, "pattern"].str.findall('>').map(len) >= 2).index.tolist()]

,pattern,supp,conf,time
6,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
7,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
8,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.698630,0.704420,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
9,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
11,Load_StatusMBALights>Load_StatusSensHeatPrntBD...,0.701370,0.707182,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
...,...,...,...,...
5733,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.704110,0.711911,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."
5734,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.989041,1.000000,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."
5735,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.846575,0.853591,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."
5736,Load_StatusSensHeatChildBUP>Load_StatusPlugLoa...,0.704110,0.711911,"{'134': [[['2013-11-12 18:29:02', '2013-11-12 ..."


In [144]:
person_appliance_level3 = level3_df_follow_restricted.loc[person_appliance_rule_indexes]
person_appliance_level3.loc[(person_appliance_level3['pattern'].str.contains('LR')) & (person_appliance_level3['pattern'].str.contains('KitchenLight')) & (person_appliance_level3['pattern'].str.contains('PrntB'))]

,pattern,supp,conf,time
1013,Load_StatusKitchenLights>Load_StatusPlugLoadLR...,0.558904,0.558904,"{'134': [[['2013-11-12 05:59:03', '2013-11-12 ..."
